# Tic-Tac-Toe Library Demo

 This notebook demonstrates how to use the Tic-Tac-Toe library to build games with different player types and configurations.


## Setup
// Import the necessary components from the library

In [15]:
import org.jetbrains.kotlinx.tictactoe.api.*
import org.jetbrains.kotlinx.tictactoe.model.*
import org.jetbrains.kotlinx.tictactoe.game.*
import org.jetbrains.kotlinx.tictactoe.dsl.*
import kotlinx.coroutines.*
import kotlin.random.Random

## Part 1: Understanding the Core Engine

 The `TicTacToeGame` class is the core game engine. It handles game logic, move validation,
 and win detection. Let's explore it directly:

In [16]:
val game = TicTacToeGame(boardSize = 3)

In [17]:
// Play some moves
game.playMove(BoardPosition(0, 0)) // X plays top-left
game.playMove(BoardPosition(1, 1)) // O plays center
game.playMove(BoardPosition(0, 1)) // X plays top-middle

In [18]:
// Get the current game state
val state = game.getState()
println("Next to play: ${state.nextToPlay}")
println("Game over: ${state.isOver}")
println("Available moves: ${state.getAvailableMoves()}")

Next to play: O
Game over: false
Available moves: [BoardPosition(row=0, column=2), BoardPosition(row=1, column=0), BoardPosition(row=1, column=2), BoardPosition(row=2, column=0), BoardPosition(row=2, column=1), BoardPosition(row=2, column=2)]


 ## Part 2: Visualizing the Board

 Let's create a helper function to display the board nicely:


In [19]:
fun printBoard(boardState: BoardState) {
    println("\n  " + (0 until boardState.size).joinToString("   "))
    println("  " + "----".repeat(boardState.size))

    for (row in 0 until boardState.size) {
        print("$row|")
        for (col in 0 until boardState.size) {
            val cell = boardState[row, col]
            val symbol = when (cell) {
                is BoardCell.Empty -> " "
                is BoardCell.Occupied -> cell.playerMark.name
            }
            print(" $symbol |")
        }
        println("\n  " + "----".repeat(boardState.size))
    }
}

// Display the current board
printBoard(game.getState().board)


  0   1   2
  ------------
0| X | X |   |
  ------------
1|   | O |   |
  ------------
2|   |   |   |
  ------------


## Part 3: Playing a Complete Game

 Let's play a complete game using the core engine to see win detection:

In [20]:
val completeGame = TicTacToeGame(boardSize = 3)

// X wins with a diagonal
completeGame.playMove(BoardPosition(0, 0)) // X
completeGame.playMove(BoardPosition(0, 1)) // O
completeGame.playMove(BoardPosition(1, 1)) // X
completeGame.playMove(BoardPosition(0, 2)) // O
completeGame.playMove(BoardPosition(2, 2)) // X wins!

val finalState = completeGame.getState()
printBoard(finalState.board)

println("\nGame over: ${finalState.isOver}")
println("Winner: ${finalState.winner}")
println("Draw: ${finalState.isDraw}")


  0   1   2
  ------------
0| X | O | O |
  ------------
1|   | X |   |
  ------------
2|   |   | X |
  ------------

Game over: true
Winner: X
Draw: false


## Part 4: Creating AI Players

 The library uses the `Player` interface for different player types.
 Let's create a random AI:

In [21]:
class RandomAI(override val name: String) : Player {
    override suspend fun selectMove(gameState: GameState): BoardPosition {
        val availableMoves = gameState.getAvailableMoves()
        require(availableMoves.isNotEmpty()) { "No available moves" }
        delay(300) // Simulate thinking time
        return availableMoves.random()
    }
}

// Create a simple AI that tries to win or block
class SmartAI(override val name: String, private val mark: PlayerMark) : Player {
    override suspend fun selectMove(gameState: GameState): BoardPosition {
        delay(300) // Simulate thinking time

        // Try to find a winning move
        val winningMove = findWinningMove(gameState, mark)
        if (winningMove != null) return winningMove

        // Try to block opponent's winning move
        val blockingMove = findWinningMove(gameState, mark.opposite())
        if (blockingMove != null) return blockingMove

        // Take center if available
        val center = BoardPosition(gameState.board.size / 2, gameState.board.size / 2)
        if (gameState.isValidMove(center)) return center

        // Otherwise, pick a random move
        return gameState.getAvailableMoves().random()
    }

    private fun findWinningMove(gameState: GameState, mark: PlayerMark): BoardPosition? {
        val tempGame = TicTacToeGame(gameState)

        for (move in gameState.getAvailableMoves()) {
            val testGame = tempGame.withMove(move)
            if (testGame.winner == mark) {
                return move
            }
        }
        return null
    }
}


 ## Part 5: Using TicTacToeGameRunner

 The `TicTacToeGameRunner` orchestrates complete games with event handling.
 Let's create a game with event logging:


In [22]:
class ConsoleEventListener : GameEventListener {
    override suspend fun onGameEvent(event: GameEvent) {
        when (event) {
            is GameEvent.BoardUpdated -> {
                println("\n--- Board Updated ---")
                printBoard(event.state.board)
                if (!event.state.isOver) {
                    println("Next player: ${event.state.nextToPlay}")
                }
            }
            is GameEvent.InvalidMove -> {
                println("❌ Invalid move: ${event.message}")
            }
            is GameEvent.GameOver -> {
                println("\n🎮 GAME OVER!")
                if (event.isDraw) {
                    println("Result: It's a draw!")
                } else {
                    println("Winner: ${event.winner} 🎉")
                }
            }
        }
    }
}
// Run a game between two AIs
runBlocking {
    val runner = TicTacToeGameRunner(
        xPlayerProvider = { RandomAI("Random Bot X") },
        oPlayerProvider = { SmartAI("Smart Bot O", PlayerMark.O) },
        listener = ConsoleEventListener(),
        boardSize = 3
    )

    println("🎲 Starting game: Random AI vs Smart AI\n")
    runner.play()
}

🎲 Starting game: Random AI vs Smart AI


--- Board Updated ---

  0   1   2
  ------------
0|   |   |   |
  ------------
1|   |   |   |
  ------------
2|   |   |   |
  ------------
Next player: X

--- Board Updated ---

  0   1   2
  ------------
0| X |   |   |
  ------------
1|   |   |   |
  ------------
2|   |   |   |
  ------------
Next player: O

--- Board Updated ---

  0   1   2
  ------------
0| X |   |   |
  ------------
1|   | O |   |
  ------------
2|   |   |   |
  ------------
Next player: X

--- Board Updated ---

  0   1   2
  ------------
0| X |   |   |
  ------------
1|   | O |   |
  ------------
2|   |   | X |
  ------------
Next player: O

--- Board Updated ---

  0   1   2
  ------------
0| X |   |   |
  ------------
1| O | O |   |
  ------------
2|   |   | X |
  ------------
Next player: X

--- Board Updated ---

  0   1   2
  ------------
0| X |   | X |
  ------------
1| O | O |   |
  ------------
2|   |   | X |
  ------------
Next player: O

--- Board Updated ---



## Part 6: Using the DSL

 The library provides a convenient DSL for game setup. This is the recommended approach
 for most use cases as it's concise and expressive:



In [23]:
runBlocking {
    val dslGame = ticTacToeGame(boardSize = 3) {
        playerX("Alice") { gameState ->
            // Human-like player that picks first available move
            delay(500) // Thinking time
            gameState.getAvailableMoves().first()
        }

        playerO("Bob") { gameState ->
            // Another strategy: prefer corners
            delay(500)
            val corners = listOf(
                BoardPosition(0, 0), BoardPosition(0, 2),
                BoardPosition(2, 0), BoardPosition(2, 2)
            )
            corners.firstOrNull { gameState.isValidMove(it) }
                ?: gameState.getAvailableMoves().random()
        }

        onEvent { event ->
            when (event) {
                is GameEvent.BoardUpdated -> {
                    if (!event.state.isOver) {
                        println("${event.state.nextToPlay}'s turn...")
                    }
                }

                is GameEvent.InvalidMove -> {
                    println("Invalid move: ${event.message}")
                }

                is GameEvent.GameOver -> {
                    println("\n✨ Game completed!")
                    println("Draw: ${event.isDraw}, Winner: ${event.winner}")
                }
            }
        }
    }

    println("\n🎮 DSL Game: Alice vs Bob\n")
    dslGame.play()
}


🎮 DSL Game: Alice vs Bob

X's turn...
O's turn...
X's turn...
O's turn...
X's turn...
O's turn...
X's turn...
O's turn...

✨ Game completed!
Draw: false, Winner: O


## Part 7: Advanced - Minimax AI

 Let's implement a perfect-play AI using the Minimax algorithm:


In [24]:
class MinimaxAI(override val name: String, private val mark: PlayerMark) : Player {
    override suspend fun selectMove(gameState: GameState): BoardPosition {
        delay(400) // Simulate thinking
        return findBestMove(gameState)
    }

    private fun findBestMove(gameState: GameState): BoardPosition {
        var bestScore = Int.MIN_VALUE
        var bestMove = gameState.getAvailableMoves().first()

        val tempGame = TicTacToeGame(gameState)

        for (move in gameState.getAvailableMoves()) {
            val testGame = tempGame.withMove(move)
            val score = minimax(testGame.getState(), 0, false)

            if (score > bestScore) {
                bestScore = score
                bestMove = move
            }
        }

        return bestMove
    }
    private fun minimax(state: GameState, depth: Int, isMaximizing: Boolean): Int {
        // Terminal states
        if (state.winner == mark) return 10 - depth
        if (state.winner == mark.opposite()) return depth - 10
        if (state.isDraw) return 0

        val tempGame = TicTacToeGame(state)

        if (isMaximizing) {
            var bestScore = Int.MIN_VALUE
            for (move in state.getAvailableMoves()) {
                val testGame = tempGame.withMove(move)
                val score = minimax(testGame.getState(), depth + 1, false)
                bestScore = maxOf(bestScore, score)
            }
            return bestScore
        } else {
            var bestScore = Int.MAX_VALUE
            for (move in state.getAvailableMoves()) {
                val testGame = tempGame.withMove(move)
                val score = minimax(testGame.getState(), depth + 1, true)
                bestScore = minOf(bestScore, score)
            }
            return bestScore
        }
    }
}

// Watch two perfect AIs play (should always draw)
runBlocking {
    val perfectGame = ticTacToeGame {
        playerX("Minimax X") { state ->
            MinimaxAI("Minimax X", PlayerMark.X).selectMove(state)
        }

        playerO("Minimax O") { state ->
            MinimaxAI("Minimax O", PlayerMark.O).selectMove(state)
        }

        onEvent { event ->
            if (event is GameEvent.BoardUpdated) {
                printBoard(event.state.board)
                delay(800) // Slow down to watch
            } else if (event is GameEvent.GameOver) {
                println("\n⚔ Battle of perfect AIs complete!")
                println("Result: ${if (event.isDraw) "Draw (as expected!)" else "Winner: ${event.winner}"}")
            }
        }
    }

    println("\n🤖 Two perfect AIs playing:\n")
    perfectGame.play()
}


🤖 Two perfect AIs playing:


  0   1   2
  ------------
0|   |   |   |
  ------------
1|   |   |   |
  ------------
2|   |   |   |
  ------------

  0   1   2
  ------------
0| X |   |   |
  ------------
1|   |   |   |
  ------------
2|   |   |   |
  ------------

  0   1   2
  ------------
0| X |   |   |
  ------------
1|   | O |   |
  ------------
2|   |   |   |
  ------------

  0   1   2
  ------------
0| X | X |   |
  ------------
1|   | O |   |
  ------------
2|   |   |   |
  ------------

  0   1   2
  ------------
0| X | X | O |
  ------------
1|   | O |   |
  ------------
2|   |   |   |
  ------------

  0   1   2
  ------------
0| X | X | O |
  ------------
1|   | O |   |
  ------------
2| X |   |   |
  ------------

  0   1   2
  ------------
0| X | X | O |
  ------------
1| O | O |   |
  ------------
2| X |   |   |
  ------------

  0   1   2
  ------------
0| X | X | O |
  ------------
1| O | O | X |
  ------------
2| X |   |   |
  ------------

  0   1   2
  ------------

## Part 8: Custom Board Sizes

 The library supports different board sizes. Let's try a 4x4 game:


In [25]:
runBlocking {
    val bigGame = ticTacToeGame(boardSize = 4) {
        playerX("Big X") { state ->
            delay(300)
            state.getAvailableMoves().random()
        }

        playerO("Big O") { state ->
            delay(300)
            state.getAvailableMoves().random()
        }

        onEvent { event ->
            if (event is GameEvent.BoardUpdated) {
                printBoard(event.state.board)
            } else if (event is GameEvent.GameOver) {
                println("\n🎯 4x4 Game complete!")
                println("Winner: ${event.winner ?: "Draw"}")
            }
        }
    }

    println("\n📐 Playing on a 4x4 board:\n")
    bigGame.play()
}


📐 Playing on a 4x4 board:


  0   1   2   3
  ----------------
0|   |   |   |   |
  ----------------
1|   |   |   |   |
  ----------------
2|   |   |   |   |
  ----------------
3|   |   |   |   |
  ----------------

  0   1   2   3
  ----------------
0|   |   |   |   |
  ----------------
1|   |   |   |   |
  ----------------
2|   |   | X |   |
  ----------------
3|   |   |   |   |
  ----------------

  0   1   2   3
  ----------------
0|   |   |   |   |
  ----------------
1|   |   |   |   |
  ----------------
2|   |   | X |   |
  ----------------
3|   |   |   | O |
  ----------------

  0   1   2   3
  ----------------
0|   |   |   |   |
  ----------------
1|   |   |   | X |
  ----------------
2|   |   | X |   |
  ----------------
3|   |   |   | O |
  ----------------

  0   1   2   3
  ----------------
0| O |   |   |   |
  ----------------
1|   |   |   | X |
  ----------------
2|   |   | X |   |
  ----------------
3|   |   |   | O |
  ----------------

  0   1   2   3
  -------------

## Summary

 This notebook demonstrated three ways to use the library:

 1. **Direct Engine Usage** (`TicTacToeGame`): Full control over game flow, ideal for
    implementing custom game loops or integrating with existing systems.

 2. **Game Runner** (`TicTacToeGameRunner`): Handles the game loop automatically with
    async support, perfect for implementing different player types with suspend functions.

 3. **DSL** (`ticTacToeGame`): Most concise and readable, recommended for quick setup
    and prototyping. Leverages Kotlin's type-safe builders.

 ### Key Features Showcased:
 - ✅ Multiple player types (Random, Smart, Minimax)
 - ✅ Event-driven architecture with `GameEventListener`
 - ✅ Coroutine support for async operations
 - ✅ Immutable game state snapshots
 - ✅ Flexible board sizes
 - ✅ Clean separation between game logic and UI

 ### Extending the Library:
 - Implement new `Player` types for different AI strategies
 - Create custom `GameEventListener` implementations for different UIs (GUI, web, etc.)
 - Use `TicTacToeGame.withMove()` for game tree analysis
 - Serialize `GameState` for save/load functionality


In [1]:
println("\n✨ Demo complete! You now know how to use the Tic-Tac-Toe library.")


✨ Demo complete! You now know how to use the Tic-Tac-Toe library.
